# Address Data Harvesting and Repair

In [1]:
#Define all imports

import pandas as pd
import pathlib as Path
import numpy as np
import re

#Required for Nominatim Address Conversion
import time
import geopandas
import geopy
from geopy.geocoders import Nominatim
locator = Nominatim(user_agent='myGeocoder')

#Required for geoplotting
import matplotlib.pyplot as plt
import folium
from folium import plugins
from shapely.geometry import Point

#Suppress warning messages
import warnings
warnings.filterwarnings('ignore')

#Allow pandas to display all data
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', None)

In [2]:
#The function takes 3 arguments: The list of addresses we want to process,
#a dictionary of existing geocode items, and a list of broken addresses that
#Nominatim was unable to handle.

def get_geodata(addresses,geocode_items,broken_addresses):
    
    #Iterate through each address
    for item in addresses:
        
        #Exclude addresses we already have data for
        if item not in (geocode_items.keys() or broken_addresses):
            
            #Define allowed number of retries for contacting Nominatim
            tries = 5
            for i in range(tries):
                
                #Attempt to get the data
                try:
                    geodata = locator.geocode(item)
                
                #If failed, retry
                except:
                    if i < tries:
                        continue
                        
                    #Stop if retry limit reached    
                    else:
                        raise
                break
                
            #Nominatim returns a None type if it was unable to provide geodata.
            #We can use this to catalogue the broken addresses
            if type(geodata) == type(None):
                broken_addresses.append(item)
                print('Broken. Appending to broken addresses')
                
            #Capture address data    
            else:
                geocode_items['Address'].append(item)
                geocode_items['Geodata'].append(geodata)
                print('Address captured')
            
            #Nominatim has a limit of 1 request per second. We will offset by 2 to ensure
            #no attempts to access the service are blocked
            time.sleep(2)

In [3]:
import re

In [4]:
import pandas as pd

broken_donation_addresses = pd.read_csv('https://raw.githubusercontent.com/mbartnik9/PDA/main/PDA_Assignment/bda.csv?token=AE7N36FAORQ44N4BJECNFQTAVSE64')
broken_donation_addresses.columns = ['Address']
broken_contract_addresses = pd.read_csv('https://raw.githubusercontent.com/mbartnik9/PDA/main/PDA_Assignment/bca.csv?token=AE7N36A3JYESVPR6WNCPBBDAVSHEA')
broken_contract_addresses.columns = ['Address']

broken_donation_addresses.head()
broken_contract_addresses.head()

,Address
0,"89 Hudson Street, Copiague, NY 11726"
1,"123 Route 46, Fairfield, NJ 07004"
2,"260 West 39th Street, Suite 302, New York, NY 10018"
3,"440 Park Ave South, 3rd Floor, New York, NY 10018"
4,"175 Rolling Hills Drive, Staten Island, NY 10312"


In [5]:
broken_donation_addresses

,Address
0,"18102 W Allen St FL, Allentown"
1,"24-08 32nd Street, Long Island City"
2,"164 27th Sreet, Brooklyn"
3,"42-09 28th Avenue, Long Island City"
4,"111 E 33 33, New York"
...,...
1022,"21 East 90th Street Apt 3C, New York"
1023,"PO Box 11st37, NYC"
1024,"481 Amoros Court, Marin County"
1025,"225 Lafayette 9B, New York"


In [6]:
broken_contract_addresses

,Address
0,"89 Hudson Street, Copiague, NY 11726"
1,"123 Route 46, Fairfield, NJ 07004"
2,"260 West 39th Street, Suite 302, New York, NY 10018"
3,"440 Park Ave South, 3rd Floor, New York, NY 10018"
4,"175 Rolling Hills Drive, Staten Island, NY 10312"
...,...
11922,"73-05 37th Road, Jackson Heights, New York 11372"
11923,"130 Marcy Place, Apt. #1-C, Bronx, NY 10452"
11924,"720 East Butterfield Road, Suite 300, Lombard, IL 60148"
11925,"143-07 101st Avenue, Jamaica, New York 11435"


In [44]:
street_types = ['(?i)Alley', 'Ally',
                'Arcade','Arc',
                'Avenue','Ave',
                'Boulevard','Bvd',
                'Bypass','Bypa',
                'Circuit','Cct',
                'Close','Cl',
                'Corner','Crn',
                'Court','Ct',
                'Crescent','Cres','Cr',
                'Cul-de-sac','Cds',
                'Drive','Dr',
                'Esplanade','Esp',
                'Green','Grn',
                'Grove','Gr',
                'Highway','Hwy',
                'Junction','Jnc',
                'Lane','Ln',
                'Link','Mews',
                'Parade','Pde',
                'Place','Pl',
                'Ridge','Rdge',
                'Road','Rd',
                'Square','Sq',
                'Street','St',
                'Terrace','Tce',
                'Way']

print("[ ,]| (?i)".join(street_types))

(?i)Alley[ ,]| (?i)Ally[ ,]| (?i)Arcade[ ,]| (?i)Arc[ ,]| (?i)Avenue[ ,]| (?i)Ave[ ,]| (?i)Boulevard[ ,]| (?i)Bvd[ ,]| (?i)Bypass[ ,]| (?i)Bypa[ ,]| (?i)Circuit[ ,]| (?i)Cct[ ,]| (?i)Close[ ,]| (?i)Cl[ ,]| (?i)Corner[ ,]| (?i)Crn[ ,]| (?i)Court[ ,]| (?i)Ct[ ,]| (?i)Crescent[ ,]| (?i)Cres[ ,]| (?i)Cr[ ,]| (?i)Cul-de-sac[ ,]| (?i)Cds[ ,]| (?i)Drive[ ,]| (?i)Dr[ ,]| (?i)Esplanade[ ,]| (?i)Esp[ ,]| (?i)Green[ ,]| (?i)Grn[ ,]| (?i)Grove[ ,]| (?i)Gr[ ,]| (?i)Highway[ ,]| (?i)Hwy[ ,]| (?i)Junction[ ,]| (?i)Jnc[ ,]| (?i)Lane[ ,]| (?i)Ln[ ,]| (?i)Link[ ,]| (?i)Mews[ ,]| (?i)Parade[ ,]| (?i)Pde[ ,]| (?i)Place[ ,]| (?i)Pl[ ,]| (?i)Ridge[ ,]| (?i)Rdge[ ,]| (?i)Road[ ,]| (?i)Rd[ ,]| (?i)Square[ ,]| (?i)Sq[ ,]| (?i)Street[ ,]| (?i)St[ ,]| (?i)Terrace[ ,]| (?i)Tce[ ,]| (?i)Way


In [21]:
street_number = re.compile('^(\d{1,}-?\d{1,})\s')
street_name = re.compile('(\S+(?= (?i)Alley[ ,]| (?i)Ally[ ,]| (?i)Arcade[ ,]| (?i)Arc[ ,]| (?i)Avenue[ ,]| (?i)Ave[ ,]| (?i)Boulevard[ ,]| (?i)Bvd[ ,]| (?i)Bypass[ ,]| (?i)Bypa[ ,]| (?i)Circuit[ ,]| (?i)Cct[ ,]| (?i)Close[ ,]| (?i)Cl[ ,]| (?i)Corner[ ,]| (?i)Crn[ ,]| (?i)Court[ ,]| (?i)Ct[ ,]| (?i)Crescent[ ,]| (?i)Cres[ ,]| (?i)Cr[ ,]| (?i)Cul-de-sac[ ,]| (?i)Cds[ ,]| (?i)Drive[ ,]| (?i)Dr[ ,]| (?i)Esplanade[ ,]| (?i)Esp[ ,]| (?i)Green[ ,]| (?i)Grn[ ,]| (?i)Grove[ ,]| (?i)Gr[ ,]| (?i)Highway[ ,]| (?i)Hwy[ ,]| (?i)Junction[ ,]| (?i)Jnc[ ,]| (?i)Lane[ ,]| (?i)Ln[ ,]| (?i)Link[ ,]| (?i)Mews[ ,]| (?i)Parade[ ,]| (?i)Pde[ ,]| (?i)Place[ ,]| (?i)Pl[ ,]| (?i)Ridge[ ,]| (?i)Rdge[ ,]| (?i)Road[ ,]| (?i)Rd[ ,]| (?i)Square[ ,]| (?i)Sq[ ,]| (?i)Street[ ,]| (?i)St[ ,]| (?i)Terrace[ ,]| (?i)Tce[ ,]| (?i)Way[ ,]))')
street_type = re.compile('((?i)Alley[ ,]| (?i)Ally[ ,]| (?i)Arcade[ ,]| (?i)Arc[ ,]| (?i)Avenue[ ,]| (?i)Ave[ ,]| (?i)Boulevard[ ,]| (?i)Bvd[ ,]| (?i)Bypass[ ,]| (?i)Bypa[ ,]| (?i)Circuit[ ,]| (?i)Cct[ ,]| (?i)Close[ ,]| (?i)Cl[ ,]| (?i)Corner[ ,]| (?i)Crn[ ,]| (?i)Court[ ,]| (?i)Ct[ ,]| (?i)Crescent[ ,]| (?i)Cres[ ,]| (?i)Cr[ ,]| (?i)Cul-de-sac[ ,]| (?i)Cds[ ,]| (?i)Drive[ ,]| (?i)Dr[ ,]| (?i)Esplanade[ ,]| (?i)Esp[ ,]| (?i)Green[ ,]| (?i)Grn[ ,]| (?i)Grove[ ,]| (?i)Gr[ ,]| (?i)Highway[ ,]| (?i)Hwy[ ,]| (?i)Junction[ ,]| (?i)Jnc[ ,]| (?i)Lane[ ,]| (?i)Ln[ ,]| (?i)Link[ ,]| (?i)Mews[ ,]| (?i)Parade[ ,]| (?i)Pde[ ,]| (?i)Place[ ,]| (?i)Pl[ ,]| (?i)Ridge[ ,]| (?i)Rdge[ ,]| (?i)Road[ ,]| (?i)Rd[ ,]| (?i)Square[ ,]| (?i)Sq[ ,]| (?i)Street[ ,]| (?i)St[ ,]| (?i)Terrace[ ,]| (?i)Tce[ ,]| (?i)Way[ ,])')
postcode_details = re.compile('(\d{5})$')



new_donation_addresses = {"Address":[],"New_Address":[]}
new_contract_addresses = {"Address":[],"New_Address":[]}


for item in broken_donation_addresses.Address:
    if item == '90-04 161stst Street, Jamaica':
        item = '90-04 161st Street, Jamaica'
    new_donation_addresses["Address"].append(item)
    new_donation_addresses["New_Address"].append(re.sub(r'[\[\]\'\"]','',((str(street_number.findall(item)) + ' ' + str(street_name.findall(item)) + ' ' + str(street_type.findall(item)) + ' ' + str(postcode_details.findall(item))))))

for item in broken_contract_addresses.Address:
    if item == '90-04 161stst Street, Jamaica':
        item = '90-04 161st Street, Jamaica'
    new_contract_addresses["Address"].append(item)
    new_contract_addresses["New_Address"].append(re.sub(r'[\[\]\'\"]','',((str(street_number.findall(item)) + ' ' + str(street_name.findall(item)) + ' ' + str(street_type.findall(item)) + ' ' + str(postcode_details.findall(item))))))



In [22]:
new_donation_addresses = pd.DataFrame.from_dict(new_donation_addresses)
new_contract_addresses = pd.DataFrame.from_dict(new_contract_addresses)

,Address,New_Address
7,"90-04 161st Street, Jamaica","90-04 161st Street,"
35,"90-04 161st Street, Jamaica","90-04 161st Street,"
64,"90-04 161st Street, Jamaica","90-04 161st Street,"
93,"90-04 161st Street, Jamaica","90-04 161st Street,"
122,"90-04 161st Street, Jamaica","90-04 161st Street,"


In [23]:
broken_donation_addresses = []
donation_geocode_items = {"Address":[], "Geodata":[]}

get_geodata(new_donation_addresses["New_Address"],donation_geocode_items,broken_donation_addresses)

Address captured
Address captured
Address captured
Address captured
Address captured
Broken. Appending to broken addresses
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Broken. Appending to broken addresses
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Broken. Appending to broken addresses
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Broken. Appending to broken addresses
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address capture

Broken. Appending to broken addresses
Broken. Appending to broken addresses
Broken. Appending to broken addresses
Broken. Appending to broken addresses
Address captured
Address captured
Address captured
Address captured
Address captured
Broken. Appending to broken addresses
Broken. Appending to broken addresses
Address captured
Address captured
Address captured
Broken. Appending to broken addresses
Address captured
Address captured
Address captured
Address captured
Broken. Appending to broken addresses
Address captured
Broken. Appending to broken addresses
Address captured
Address captured
Address captured
Broken. Appending to broken addresses
Broken. Appending to broken addresses
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Broken. Appending to broken addresses


Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Broken. Appending to broken addresses
Broken. Appending to broken addresses
Address captured
Address captured
Address captured
Address captured
Broken. Appending to broken addresses
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Broken. Appending to broken addresses
Address captured
Address captured
Address captured
Broken. Appending to broken addresses
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Broken. Appending to broken addresses
Broken. Appending to broken addresses
Address captured
Broken. Appending to broken addresses
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured
Address captured

In [ ]:
broken_contract_addresses = []
contract_geocode_items = {"Address":[], "Geodata":[]}

get_geodata(new_contract_addresses["New_Address"],contract_geocode_items,broken_contract_addresses)

In [24]:
donation_geocode_addresses = pd.DataFrame.from_dict(donation_geocode_items)

donation_geocode_addresses = donation_geocode_addresses[donation_geocode_addresses.Address.str.contains(r'[A-Za-z]')]

donation_geocode_addresses

,Address,Geodata
0,18102 Allen St,"(The Strand, Allen, Collin County, Texas, United States, (33.1325838, -96.70203420493834))"
1,"24-08 32nd Street,","(24, 32nd Street, Copiague, Suffolk County, New York, 11726, United States, (40.68987369230769, -73.39522138461538))"
2,164,"(Cmentarz wojenny nr 164 – Tuchów, Furmaniec, Tuchów, gmina Tuchów, powiat tarnowski, województwo małopolskie, Polska, (49.905195, 21.08427125))"
3,"42-09 28th Avenue,","(42-09, 28th Avenue, Queens, Queens County, New York, 11103, United States, (40.764713650000004, -73.91162700000001))"
4,111,"(Neo Química Arena, 111, Avenida Doutor Luís Ayres, Cidade Antônio Estêvão de Carvalho, Itaquera, São Paulo, Região Imediata de São Paulo, Região Metropolitana de São Paulo, Região Geográfica Intermediária de São Paulo, São Paulo, Região Sudeste, 08295-005, Brasil, (-23.54531185, -46.4743121482021))"
...,...,...
832,"881 Polite Avenue,","(Polite Avenue, Trilby, Pasco County, Florida, 33593, United States, (28.457184, -82.19898))"
833,21 90th Street,"(97-21, 90th Street, Woodhaven, Queens, Queens County, New York, 11416, United States, (40.68417375, -73.85060244444924))"
834,"481 Amoros Court,","(481, Amoros Court, Santa Rosa, Sonoma County, California, 95401, United States, (38.4348262, -122.7637934))"
835,225,"(みずほ銀行, 千川通り, 練馬区, 東京都, 176-0002, 日本, (35.7383371, 139.6615041))"


In [25]:
new_donation_addresses = pd.DataFrame(new_donation_addresses)
new_donation_addresses = new_donation_addresses.rename(columns = {"Address":"Old_Address", "New_Address":"Address"})

address_mapping = dict(new_donation_addresses[['Address', 'Old_Address']].values)


In [27]:
donation_geocode_addresses

,Address,Geodata
0,18102 Allen St,"(The Strand, Allen, Collin County, Texas, United States, (33.1325838, -96.70203420493834))"
1,"24-08 32nd Street,","(24, 32nd Street, Copiague, Suffolk County, New York, 11726, United States, (40.68987369230769, -73.39522138461538))"
3,"42-09 28th Avenue,","(42-09, 28th Avenue, Queens, Queens County, New York, 11103, United States, (40.764713650000004, -73.91162700000001))"
5,"Verizon Way,","(Verizon Way, Van Dorans Mills, Bernards Township, Somerset County, New Jersey, 02794, United States, (40.725639, -74.535133))"
6,"90-04 161st Street,","(90-04, 161st Street, Jamaica, Queens, Queens County, New York, 11432, United States, (40.7052217, -73.7994126))"
...,...,...
830,400 43rd St,"(400, 43rd Street, Lindenhurst, Suffolk County, New York, 11757, United States, (40.697277, -73.392061))"
832,"881 Polite Avenue,","(Polite Avenue, Trilby, Pasco County, Florida, 33593, United States, (28.457184, -82.19898))"
833,21 90th Street,"(97-21, 90th Street, Woodhaven, Queens, Queens County, New York, 11416, United States, (40.68417375, -73.85060244444924))"
834,"481 Amoros Court,","(481, Amoros Court, Santa Rosa, Sonoma County, California, 95401, United States, (38.4348262, -122.7637934))"


In [42]:
donation_geocode_addresses['Old_Address'] = donation_geocode_addresses.Address.map(address_mapping)

def geodata_lat(row):
    return row['Geodata'].latitude

def geodata_long(row):
    return row['Geodata'].longitude

def geodata_address(row):
    return row['Geodata'].address


donation_geocode_addresses['GeoAddress'] = donation_geocode_addresses.apply(lambda row: geodata_address(row), axis=1)

donation_geocode_addresses['Latitude'] = donation_geocode_addresses.apply(lambda row: geodata_lat(row), axis=1)

donation_geocode_addresses['Longitude'] = donation_geocode_addresses.apply(lambda row: geodata_long(row), axis=1)

donation_geocode_addresses = donation_geocode_addresses.drop(columns=["Geodata"])

donation_geocode_addresses.head()

,Address,Old_Address,GeoAddress,Latitude,Longitude
0,18102 Allen St,"18102 W Allen St FL, Allentown","The Strand, Allen, Collin County, Texas, United States",33.132584,-96.702034
1,"24-08 32nd Street,","24-08 32nd Street, Long Island City","24, 32nd Street, Copiague, Suffolk County, New York, 11726, United States",40.689874,-73.395221
3,"42-09 28th Avenue,","42-09 28th Avenue, Long Island City","42-09, 28th Avenue, Queens, Queens County, New York, 11103, United States",40.764714,-73.911627
5,"Verizon Way,","1 Verizon Way, Basking Ridge","Verizon Way, Van Dorans Mills, Bernards Township, Somerset County, New Jersey, 02794, United States",40.725639,-74.535133
6,"90-04 161st Street,","90-04 161st Street, Jamaica","90-04, 161st Street, Jamaica, Queens, Queens County, New York, 11432, United States",40.705222,-73.799413


In [43]:
donation_geocode_addresses.to_csv('donation_fixed_addresses.csv', index=False)

**Repairing Contract Addresses**

In [ ]:
contract_geocode_addresses = pd.DataFrame.from_dict(contract_geocode_items)

contract_geocode_addresses = contract_geocode_addresses[contract_geocode_addresses.Address.str.contains(r'[A-Za-z]')]

contract_geocode_addresses

In [ ]:
new_contract_addresses = pd.DataFrame(new_contract_addresses)
new_contract_addresses = new_contract_addresses.rename(columns = {"Address":"Old_Address", "New_Address":"Address"})

address_mapping = dict(new_contract_addresses[['Address', 'Old_Address']].values)


In [ ]:
contract_geocode_addresses['Old_Address'] = contract_geocode_addresses.Address.map(address_mapping)

contract_geocode_addresses['GeoAddress'] = contract_geocode_addresses.apply(lambda row: geodata_address(row), axis=1)

contract_geocode_addresses['Latitude'] = contract_geocode_addresses.apply(lambda row: geodata_lat(row), axis=1)

contract_geocode_addresses['Longitude'] = contract_geocode_addresses.apply(lambda row: geodata_long(row), axis=1)

contract_geocode_addresses = contract_geocode_addresses.drop(columns=["Geodata"])

contract_geocode_addresses.head()

In [ ]:
contract_geocode_addresses.to_csv('contract_fixed_addresses.csv', index=False)